# visualize

In [ ]:
from matplotlib import pyplot as plt
import torch
import os
import numpy as np
import cv2
import math

In [ ]:
# get var threshold
def get_var_threshold(channel_num, image):
    channel_num = channel_num
    image = image
    var_ = []
    for i in range(channel_num):
        var = image[i].var()
        var_.append(var)
    var_threshold = np.array(var_).mean()
    return var_threshold

# get coordinates
def get_coordinates(channel_num, image_size, image):
    channel_num = channel_num
    image_size = image_size
    image = image
    keypoint_coor = []
    # var_threshold = get_var_threshold(channel_num=channel_num, image=image)
    for channel in range(channel_num):
        if image[channel].mean() == 0:
            keypoint = (0,0)
        else:
            min_index = np.argmin(image[channel])
            x, y = np.unravel_index(min_index, image[channel].shape)
            keypoint = (x,y)
        keypoint_coor.append(keypoint)
    keypoint_coor = [(y, x) for x, y in keypoint_coor]
    return keypoint_coor

# get_subsets
def get_subsets(channel_num, image_size, image):
    channel_num = channel_num
    image_size = image_size
    image = image
    subset = []
    for channel in range(channel_num):
        if image[channel].mean()==0:
            subset.append(-1)
        else:
            subset.append(channel)
    return subset

# draw body
def draw_bodypose(candidate, subset):
    stickwidth = 4
    candidate = candidate * 512/64
    canvas_shape = np.zeros((512, 512, 3))
    limbSeq = [[1, 2], [1, 3], [2, 4], [3, 5], [6, 7], [6, 8], [6, 12], [7, 9], \
               [7, 13], [8, 10], [9, 11], [12, 13], [12, 14], [13, 15], [14, 16], \
               [15, 17], [4, 6], [5, 7]]

    colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
              [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
              [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]
    canvas = np.zeros_like(canvas_shape)
    for i in range(17):
        for n in range(len(subset)):
            index = int(subset[n][i])
            if index == -1:
                continue
            x, y = candidate[index][0:2]
            cv2.circle(canvas, (int(x), int(y)), 4, colors[i], thickness=-1)
    for i in range(16):
        for n in range(len(subset)):
            index = subset[n][np.array(limbSeq[i]) - 1]
            if -1 in index:
                continue
            cur_canvas = canvas.copy()
            Y = candidate[index.astype(int), 0]
            X = candidate[index.astype(int), 1]
            mX = np.mean(X)
            mY = np.mean(Y)
            length = ((X[0] - X[1]) ** 2 + (Y[0] - Y[1]) ** 2) ** 0.5
            angle = math.degrees(math.atan2(X[0] - X[1], Y[0] - Y[1]))
            polygon = cv2.ellipse2Poly((int(mY), int(mX)), (int(length / 2), stickwidth), int(angle), 0, 360, 1)
            cv2.fillConvexPoly(cur_canvas, polygon, colors[i])
            canvas = cv2.addWeighted(canvas, 0.4, cur_canvas, 0.6, 0)
    return canvas

In [ ]:
i = 0

In [ ]:
import json
file_path = '/data/train.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
image = np.load(data[i]['image_path'])
text = data[i]['text']
print("i:",i)
print("text:",text)
image = image.reshape(17,64,64)
keypoint_coor = get_coordinates(channel_num=17, image_size=32, image=image)
subset = get_subsets(channel_num=17, image_size=32, image=image)
# draw pose
canvas = draw_bodypose(np.array(keypoint_coor), np.array([subset]))
canvas = np.array((canvas - np.min(canvas)) / (np.max(canvas) - np.min(canvas)))
plt.imshow(canvas[:, :, [2, 1, 0]])
i = i+1